In [1]:
from geopy.distance import great_circle
from geopy.point import Point

base_point = Point(39.9050, 116.4053)  # 北京市的(纬(lat), 经(lon))
distance_meters = 100.0

# 使用球面三角法计算目标点的经纬度
north_destination = great_circle(meters=distance_meters).destination(base_point, 0)  # 向北100米
east_destination = great_circle(meters=distance_meters).destination(base_point, 90)

# 北京市经向/纬向移动100米产生的经/纬度变化
D_LAT = north_destination.latitude - base_point.latitude
D_LON = east_destination.longitude - base_point.longitude

In [2]:
print(f"南北纬度变化：{D_LAT}")
print(f"东西经度变化：{D_LON}")

南北纬度变化：0.0008993203354847878
东西经度变化：0.001172349866791933


**input**: 轨迹节点的GPS经纬度$p_i(x_i, y_i)$
**param**: 矩形范围的半边长经纬度$d_{lon}$和$d_{lat}$
**output**: $p_i$的候选点集$C\{c_i^1, ..., c_i^n\}$

In [3]:
import pandas as pd
from geopy.distance import geodesic  # 计算两给定经纬度点之间的球面距离

nodes = pd.read_csv("data/road_node.csv")
nodes

,coordinate
0,"(116.3894407, 39.9062721)"
1,"(116.3894463, 39.9060115)"
2,"(116.386428, 39.9061687)"
3,"(116.3856338, 39.9061421)"
4,"(116.3930703, 39.906394)"
...,...
49725,"(116.4061092, 39.8315409)"
49726,"(116.4060903, 39.8321389)"
49727,"(116.4060871, 39.8328152)"
49728,"(116.406125, 39.833363)"


In [4]:
edges = pd.read_csv("data/road1.csv")
edges

,old_id,coordinate1,coordinate2,coordinate1_lon,coordinate1_lat,coordinate2_lon,coordinate2_lat,dis
0,0,"(116.3894407, 39.9062721)","(116.3894463, 39.9060115)",116.389441,39.906272,116.389446,39.906011,28.939117
1,1,"(116.3894407, 39.9062721)","(116.386428, 39.9061687)",116.389441,39.906272,116.386428,39.906169,257.873361
2,1,"(116.386428, 39.9061687)","(116.3856338, 39.9061421)",116.386428,39.906169,116.385634,39.906142,67.976743
3,2,"(116.3930703, 39.906394)","(116.3894407, 39.9062721)",116.393070,39.906394,116.389441,39.906272,310.663605
4,3,"(116.3970962, 39.9065222)","(116.3930703, 39.906394)",116.397096,39.906522,116.393070,39.906394,344.549935
...,...,...,...,...,...,...,...,...
85390,38025,"(116.4062083, 39.8313723)","(116.406357, 39.8311645)",116.406208,39.831372,116.406357,39.831164,26.350831
85391,38026,"(116.3121472, 39.9935096)","(116.3121727, 39.9931557)",116.312147,39.993510,116.312173,39.993156,39.355411
85392,38026,"(116.3121727, 39.9931557)","(116.3121191, 39.9930952)",116.312173,39.993156,116.312119,39.993095,8.128968
85393,38026,"(116.3121191, 39.9930952)","(116.3121553, 39.9926498)",116.312119,39.993095,116.312155,39.992650,49.551302


In [5]:
def get_sub_edge_set(node_p, edge_set):
    """
    用于获取边长为200m正方形内的所有边的函数
    :param node_p: GPS轨迹点
    :param edge_set: 路网edge集
    :return: 边长为200m正方形内的所有边
    """
    global D_LON, D_LAT
    left_lon_limit = node_p[0] - D_LON
    right_lon_limit = node_p[0] + D_LON
    down_lat_limit = node_p[1] - D_LAT
    up_lat_limit = node_p[1] + D_LAT
    return edge_set[(left_lon_limit <= edge_set['coordinate1_lon']) & (edge_set['coordinate1_lon']  <= right_lon_limit) & (down_lat_limit <= edge_set['coordinate1_lat']) & (edge_set['coordinate1_lat'] <= up_lat_limit)
                    | (left_lon_limit <= edge_set['coordinate2_lon']) & (edge_set['coordinate2_lon'] <= right_lon_limit) & (down_lat_limit <= edge_set['coordinate2_lat']) & (edge_set['coordinate2_lat'] <= up_lat_limit) ]

sub_edge_set1 = get_sub_edge_set((116.3894407, 39.9062721), edges)
sub_edge_set1

,old_id,coordinate1,coordinate2,coordinate1_lon,coordinate1_lat,coordinate2_lon,coordinate2_lat,dis
0,0,"(116.3894407, 39.9062721)","(116.3894463, 39.9060115)",116.389441,39.906272,116.389446,39.906011,28.939117
1,1,"(116.3894407, 39.9062721)","(116.386428, 39.9061687)",116.389441,39.906272,116.386428,39.906169,257.873361
3,2,"(116.3930703, 39.906394)","(116.3894407, 39.9062721)",116.393070,39.906394,116.389441,39.906272,310.663605
7558,2832,"(116.3894463, 39.9060115)","(116.3930805, 39.9061466)",116.389446,39.906011,116.393080,39.906147,311.124945
7559,2833,"(116.3894463, 39.9060115)","(116.3894579, 39.9057692)",116.389446,39.906011,116.389458,39.905769,26.921533
7560,2833,"(116.3894579, 39.9057692)","(116.3895611, 39.9036108)",116.389458,39.905769,116.389561,39.903611,239.815626
9266,3550,"(116.3881308, 39.9059712)","(116.3894463, 39.9060115)",116.388131,39.905971,116.389446,39.906011,112.578478


In [6]:
from geopy.distance import geodesic
def get_candidate_nodes(node_p, sub_edge_set):
    """
    对于给定的GPS轨迹点求其候选点集的函数
    :param node_p: GPS轨迹点，(务必以(lon, lat)的形式传入)
    :param sub_edge_set: 由`get_sub_edge_set`函数求得的邻近边集
    :return: list类型，候选点的id序列
    """
    def cal_dis(row):
        p = (node_p[1], node_p[0])
        p1 = (row['coordinate1_lat'], row['coordinate1_lon'])
        p2 = (row['coordinate2_lat'], row['coordinate2_lon'])
        dist1 = geodesic(p, p1).meters
        dist2 = geodesic(p, p2).meters
        if dist1 < dist2:
            row['min_dist'] = dist1
            row['matched_node'] = 1
        else:
            row['min_dist'] = dist2
            row['matched_node'] = 2
        return row
    sub_edge_set = sub_edge_set.apply(lambda row: cal_dis(row), axis=1)
    res = []
    for old_id, group in sub_edge_set.groupby(by='old_id'):
        min_index = group['min_dist'].idxmin()
        row = group.loc[min_index]
        if row['matched_node'] == 1:
            res.append(row['coordinate1'])
        else:
            res.append(row['coordinate2'])
    return list(set(res))

In [7]:
candidate_set = get_candidate_nodes((116.3970859, 39.9067276), sub_edge_set1)
node_ids = nodes[nodes['coordinate'].isin(candidate_set)].index.tolist()
node_ids

[0, 1, 4, 7572]